In [566]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from collections import Counter

books = pd.read_csv('storygraph.csv', parse_dates = ['Date Added', 'Last Date Read'])
pd.set_option('display.max_columns', 23)

In [567]:
books.head()

,Title,Authors,Contributors,ISBN/UID,Format,Read Status,Date Added,Last Date Read,Dates Read,Read Count,Moods,Pace,Character- or Plot-Driven?,Strong Character Development?,Loveable Characters?,Diverse Characters?,Flawed Characters?,Star Rating,Review,Content Warnings,Content Warning Description,Tags,Owned?
0,The Wise Man's Fear,Patrick Rothfuss,NaN,9780756404734,hardcover,to-read,2022-08-05,NaT,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
1,A Room of One's Own,Virginia Woolf,NaN,9780141183534,paperback,to-read,2022-08-05,NaT,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No
2,The Wind-Up Bird Chronicle,Haruki Murakami,Jay Rubin,9780965341981,paperback,read,2022-06-17,NaT,NaN,1,reflective,slow,Character,No,It's complicated,No,It's complicated,2.5,NaN,NaN,NaN,"mysterious, slow-paced, long, literary",No
3,The Toll,Neal Shusterman,NaN,9781481497077,paperback,read,2022-08-02,2019-12-04,2019/12/04,1,adventurous,medium,A mix,Yes,Yes,Yes,Yes,4.0,NaN,NaN,NaN,"medium-paced, sci-fi, adventurous, long",No
4,A Certain Hunger,Chelsea G. Summers,NaN,9781951213435,paperback,to-read,2022-08-05,NaT,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No


In [568]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 831 entries, 0 to 830
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Title                          831 non-null    object        
 1   Authors                        830 non-null    object        
 2   Contributors                   153 non-null    object        
 3   ISBN/UID                       722 non-null    object        
 4   Format                         766 non-null    object        
 5   Read Status                    831 non-null    object        
 6   Date Added                     831 non-null    datetime64[ns]
 7   Last Date Read                 290 non-null    datetime64[ns]
 8   Dates Read                     290 non-null    object        
 9   Read Count                     831 non-null    int64         
 10  Moods                          447 non-null    object        
 11  Pace               

We still need to convert 'Dates Read' to datetime. Since parse_dates failed to do so when I tried, we will do it sepparately,
then check which value is causing the issue and if there is more than 1.

In [569]:
books['Dates Read'] = pd.to_datetime(books['Dates Read'], errors = 'coerce')

In [570]:
books['Dates Read'].notnull().sum()

289

We need to drop some colums that aren't useful. We'll check the time interval in which books have been added, and if it's a 
short one, we can drop 'Date Added'. We'll also compare 'Last Date Read' with 'Dates Read', because they seem to be very
similar at first glance. If they are, we'll drop one of them too.

In [571]:
books['Date Added'].max() - books['Date Added'].min()

Timedelta('53 days 00:00:00')

In [572]:
df = books.loc[books['Last Date Read'].notnull()]
df[['Last Date Read', 'Dates Read']]

,Last Date Read,Dates Read
3,2019-12-04,2019-12-04
5,2022-04-10,2022-04-10
9,2020-03-05,2020-03-05
10,2020-01-20,2020-01-20
12,2019-01-01,2019-01-01
...,...,...
816,2020-06-07,2020-06-07
818,2020-01-18,2020-01-18
820,2020-03-02,2020-03-02
825,2019-01-01,2019-01-01


In [573]:
books.drop(['ISBN/UID', 'Format', 'Date Added', 'Dates Read', 'Read Count', 'Review', 'Content Warnings',
                     'Content Warning Description', 'Owned?'], axis=1, inplace = True)

Making a new dataframe with only the read books.

In [574]:
filt = books['Read Status'] == 'read'
read = books[filt].copy()
to_read = books[~filt]
read

,Title,Authors,Contributors,Read Status,Last Date Read,Moods,Pace,Character- or Plot-Driven?,Strong Character Development?,Loveable Characters?,Diverse Characters?,Flawed Characters?,Star Rating,Tags
2,The Wind-Up Bird Chronicle,Haruki Murakami,Jay Rubin,read,NaT,reflective,slow,Character,No,It's complicated,No,It's complicated,2.50,"mysterious, slow-paced, long, literary"
3,The Toll,Neal Shusterman,NaN,read,2019-12-04,adventurous,medium,A mix,Yes,Yes,Yes,Yes,4.00,"medium-paced, sci-fi, adventurous, long"
5,Transcendent Kingdom,Yaa Gyasi,NaN,read,2022-04-10,emotional,medium,Character,Yes,Yes,Yes,Yes,4.00,"medium-paced, emotional, medium, literary"
9,Hollow City,Ransom Riggs,NaN,read,2020-03-05,adventurous,medium,A mix,Yes,Yes,Yes,No,3.75,"fantasy, medium-paced, adventurous, medium"
10,The Astonishing Color of After,Emily X.R. Pan,NaN,read,2020-01-20,emotional,medium,Character,Yes,Yes,Yes,Yes,4.50,"medium-paced, contemporary, emotional, medium"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
820,Truly Devious,Maureen Johnson,NaN,read,2020-03-02,mysterious,medium,Plot,Yes,It's complicated,Yes,Yes,3.00,"mysterious, thriller, medium-paced, medium"
823,Domniṣoara Christina,Mircea Eliade,NaN,read,NaT,mysterious,medium,Plot,No,It's complicated,No,No,4.00,"classics, mysterious, short, medium-paced"
825,Into the Drowning Deep,Mira Grant,NaN,read,2019-01-01,dark,medium,Plot,Yes,Yes,Yes,No,3.75,"dark, medium-paced, horror, medium"
826,Down Among the Sticks and Bones,Seanan McGuire,NaN,read,2019-01-01,dark,fast,A mix,Yes,Yes,Yes,Yes,5.00,"fantasy, dark, short, fast-paced"


In [575]:
read.isnull().sum()

Title                              0
Authors                            1
Contributors                     385
Read Status                        0
Last Date Read                   173
Moods                             16
Pace                              13
Character- or Plot-Driven?        29
Strong Character Development?     29
Loveable Characters?              29
Diverse Characters?               29
Flawed Characters?                29
Star Rating                       12
Tags                              10
dtype: int64

There are a lot of null values that need to be replaced, so we'll take a look at them and see wht we can do.

In [576]:
read.loc[read['Authors'].isnull()]

,Title,Authors,Contributors,Read Status,Last Date Read,Moods,Pace,Character- or Plot-Driven?,Strong Character Development?,Loveable Characters?,Diverse Characters?,Flawed Characters?,Star Rating,Tags
378,The Epic of Gilgamesh,NaN,N. K. Sandars (Translator),read,2021-07-24,adventurous,fast,Plot,No,No,No,Yes,3.0,"classics, short, adventurous, fast-paced"


In [577]:
read['Authors'].fillna('Unknown', inplace=True)
read.loc[378, 'Authors']

'Unknown'

In [578]:
read['Contributors'].fillna('Missing', inplace = True)
read['Contributors']

2      Jay Rubin
3        Missing
5        Missing
9        Missing
10       Missing
         ...    
820      Missing
823      Missing
825      Missing
826      Missing
830      Missing
Name: Contributors, Length: 463, dtype: object

Contributors are usually translators. In this case, we'll replace the null values with 'Missing'.

In [579]:
read['Last Date Read'].fillna(pd.to_datetime('2018-01-01'), inplace=True)

The null values in 'Last Date Read' are books read before 2019. I'll use a date in 2018 to replace the null values, because 
there is no way to guess the actual read dates for all these books and the important thing is for them to appear as read before
2019.

In [580]:
read.loc[read['Moods'].isnull()]

,Title,Authors,Contributors,Read Status,Last Date Read,Moods,Pace,Character- or Plot-Driven?,Strong Character Development?,Loveable Characters?,Diverse Characters?,Flawed Characters?,Star Rating,Tags
68,The Order of Time,Carlo Rovelli,Missing,read,2022-07-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"informative, nonfiction, slow-paced"
176,Brief Answers to the Big Questions,Stephen Hawking,Missing,read,2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,21 Lessons for the 21st Century,Yuval Noah Harari,Missing,read,2022-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,A Brief History of Time,Stephen Hawking,Missing,read,2020-08-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,The Diary of a Young Girl,Anne Frank,B.M. Mooyaart-Doubleday (Translator),read,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,Paula,Isabel Allende,Margaret Sayers Peden,read,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276,Beneath the Sugar Sky,Seanan McGuire,Missing,read,2019-01-01,NaN,fast,A mix,Yes,Yes,Yes,Yes,4.5,"fantasy, short, adventurous, fast-paced"
322,The Man Who Loved Books Too Much: The True Sto...,Allison Hoover Bartlett,Missing,read,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
385,Atomic Habits: An Easy & Proven Way to Build G...,James Clear,Missing,read,2022-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424,My Body,Emily Ratajkowski,Missing,read,2022-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [581]:
read['Moods'].fillna('Missing', inplace=True)
read.loc[read['Moods'] == 'Missing']

,Title,Authors,Contributors,Read Status,Last Date Read,Moods,Pace,Character- or Plot-Driven?,Strong Character Development?,Loveable Characters?,Diverse Characters?,Flawed Characters?,Star Rating,Tags
68,The Order of Time,Carlo Rovelli,Missing,read,2022-07-18,Missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"informative, nonfiction, slow-paced"
176,Brief Answers to the Big Questions,Stephen Hawking,Missing,read,2019-01-01,Missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,21 Lessons for the 21st Century,Yuval Noah Harari,Missing,read,2022-05-31,Missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,A Brief History of Time,Stephen Hawking,Missing,read,2020-08-30,Missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,The Diary of a Young Girl,Anne Frank,B.M. Mooyaart-Doubleday (Translator),read,2018-01-01,Missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,Paula,Isabel Allende,Margaret Sayers Peden,read,2018-01-01,Missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276,Beneath the Sugar Sky,Seanan McGuire,Missing,read,2019-01-01,Missing,fast,A mix,Yes,Yes,Yes,Yes,4.5,"fantasy, short, adventurous, fast-paced"
322,The Man Who Loved Books Too Much: The True Sto...,Allison Hoover Bartlett,Missing,read,2018-01-01,Missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
385,Atomic Habits: An Easy & Proven Way to Build G...,James Clear,Missing,read,2022-06-01,Missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424,My Body,Emily Ratajkowski,Missing,read,2022-04-15,Missing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [582]:
read.loc[276, 'Moods'] = 'adventurous'
read.loc[464, 'Moods'] = 'reflective'
read.loc[616, 'Moods'] = 'emotional'

Most of these are nonfiction books that don't even have a rating. Given the fact that we can't exacly talk about character 
development or lovable characters in this case, I'll drop most of these. I'll keep those that are fiction and rated, and replace
the missing mood values with the appropriate moods.

In [583]:
read.drop(read[read['Moods'] == 'Missing'].index, inplace=True)

In [584]:
read.loc[read['Character- or Plot-Driven?'].isnull()]

,Title,Authors,Contributors,Read Status,Last Date Read,Moods,Pace,Character- or Plot-Driven?,Strong Character Development?,Loveable Characters?,Diverse Characters?,Flawed Characters?,Star Rating,Tags
57,Beyond the Aquila Rift,Alastair Reynolds,Tom Dheere,read,2021-01-01,dark,fast,NaN,NaN,NaN,NaN,NaN,4.00,"dark, short, sci-fi, fast-paced"
62,Șarpele,Mircea Eliade,Missing,read,2018-01-01,reflective,medium,NaN,NaN,NaN,NaN,NaN,3.00,"classics, short, medium-paced, reflective"
119,Hanu-Ancuței,Mihail Sadoveanu,Missing,read,2018-01-01,adventurous,medium,NaN,NaN,NaN,NaN,NaN,1.00,"classics, medium-paced, adventurous, medium"
159,Iubim,Octav Dessila,Missing,read,2018-01-01,emotional,slow,NaN,NaN,NaN,NaN,NaN,4.25,"emotional, slow-paced, long, literary"
196,The Sun and Her Flowers,Rupi Kaur,Missing,read,2019-01-01,emotional,medium,NaN,NaN,NaN,NaN,NaN,3.00,"emotional, medium"
302,Two on a Tower (English Edition),Thomas Hardy,Missing,read,2018-01-01,emotional,medium,NaN,NaN,NaN,NaN,NaN,1.00,"medium-paced, emotional, medium, romance"
369,Stung with Love: Poems and Fragments,Sappho,"Carol Ann Duffy (Contributor), Aaron Poochigia...",read,2021-02-12,reflective,medium,NaN,NaN,NaN,NaN,NaN,3.75,"classics, short, reflective"
407,Viata ca o prada,Marin Preda,Missing,read,2018-01-01,reflective,medium,NaN,NaN,NaN,NaN,NaN,1.00,"classics, medium-paced, reflective, medium"
419,Aripi de zapada,Constantin Chiriță,Missing,read,2018-01-01,adventurous,medium,NaN,NaN,NaN,NaN,NaN,4.50,"classics, medium-paced, adventurous, medium"
466,LA BIBLE PERDUE,Igor Bergler,Missing,read,2018-01-01,mysterious,medium,NaN,NaN,NaN,NaN,NaN,3.50,"mysterious, thriller, medium-paced, long"


I can't drop these ones because they're important so I'll try to replace them using SimpleImputer from sklearn. There are 2
nonfiction books here too: Letters to a young poet and The Symposium, plus two books of poetry. I thought about dropping
these as well, but they have been rated and these ones felt almost like fiction(they can be considered classics), so I've
decided to keep them.  

In [585]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
read.iloc[:, 7:12] = imputer.fit_transform(read.iloc[:, 7:12])

After imputing null values, I want to take a look at the lowest rated ones. One thing all of my lowest rated books have in 
common is the fact that I didn't like the characters, so I want to make sure that all of them have 'No' as answer for
'Loveable Characters?'

In [586]:
read.loc[119, 'Loveable Characters?'] = 'No'
read.loc[302, 'Loveable Characters?'] = 'No'
read.loc[407, 'Loveable Characters?'] = 'No'

Next I want to take a look at the 'Tags' column and separate it. 

In [587]:
from collections import Counter
check = Counter()
for i in read['Tags']:
    check.update(i.split(','))
descriptions = dict(check)
descriptions

{'mysterious': 34,
 ' slow-paced': 99,
 ' long': 74,
 ' literary': 16,
 'medium-paced': 67,
 ' sci-fi': 56,
 ' adventurous': 160,
 ' emotional': 65,
 ' medium': 287,
 'fantasy': 115,
 ' medium-paced': 183,
 ' contemporary': 38,
 'sci-fi': 21,
 ' fast-paced': 98,
 ' historical': 24,
 'classics': 112,
 ' short': 63,
 ' reflective': 58,
 'dark': 47,
 'emotional': 4,
 ' fantasy': 7,
 ' thriller': 11,
 'short': 26,
 'funny': 7,
 ' horror': 27,
 ' lighthearted': 7,
 ' tense': 5,
 ' mysterious': 8,
 ' dark': 29,
 ' funny': 9,
 ' hopeful': 5,
 ' romance': 8,
 ' sad': 4,
 'reflective': 3,
 ' challenging': 2,
 'contemporary': 5,
 'historical': 2,
 ' informative': 2,
 'thriller': 6,
 ' graphic-novel': 1,
 'adventurous': 1}

Checking for whitespace, which most of them seem to have.

In [588]:
t = read['Tags'].str.split(',', expand=True)
t

,0,1,2,3
2,mysterious,slow-paced,long,literary
3,medium-paced,sci-fi,adventurous,long
5,medium-paced,emotional,medium,literary
9,fantasy,medium-paced,adventurous,medium
10,medium-paced,contemporary,emotional,medium
...,...,...,...,...
820,mysterious,thriller,medium-paced,medium
823,classics,mysterious,short,medium-paced
825,dark,medium-paced,horror,medium
826,fantasy,dark,short,fast-paced


In [589]:
t.rename(columns = {0: 'sizes', 1: 'genre', 2: 'tone', 3: 'pace'}, inplace=True)
t

,sizes,genre,tone,pace
2,mysterious,slow-paced,long,literary
3,medium-paced,sci-fi,adventurous,long
5,medium-paced,emotional,medium,literary
9,fantasy,medium-paced,adventurous,medium
10,medium-paced,contemporary,emotional,medium
...,...,...,...,...
820,mysterious,thriller,medium-paced,medium
823,classics,mysterious,short,medium-paced
825,dark,medium-paced,horror,medium
826,fantasy,dark,short,fast-paced


In [590]:
t['sizes'] = t['sizes'].str.strip()
t['genre'] = t['genre'].str.strip()
t['tone'] = t['tone'].str.strip()
t['pace'] = t['pace'].str.strip()

Moving the values so they correspond to their column names. The first two columns are important, the other 2 contain values
that are already present in the dataset.

In [591]:
t['c_sizes'] = t['sizes']

In [592]:
s = ['short', 'long', 'medium']
g = ['classics', 'contemporary', 'fantasy', 'historical', 'horror', 'literary', 'romance', 'sci-fi', 'thriller']

t['sizes'] = np.where(t['genre'].isin(s), t['genre'], t['sizes'])
t['sizes'] = np.where(t['tone'].isin(s), t['tone'], t['sizes'])
t['sizes'] = np.where(t['pace'].isin(s), t['pace'], t['sizes'])

t['genre'] = np.where(t['tone'].isin(g), t['tone'], t['genre'])
t['genre'] = np.where(t['pace'].isin(g), t['pace'], t['genre'])
t['genre'] = np.where(t['c_sizes'].isin(g), t['c_sizes'], t['genre'])


t

,sizes,genre,tone,pace,c_sizes
2,long,literary,long,literary,mysterious
3,long,sci-fi,adventurous,long,medium-paced
5,medium,literary,medium,literary,medium-paced
9,medium,fantasy,adventurous,medium,fantasy
10,medium,contemporary,emotional,medium,medium-paced
...,...,...,...,...,...
820,medium,thriller,medium-paced,medium,mysterious
823,short,classics,short,medium-paced,classics
825,medium,horror,horror,medium,dark
826,short,fantasy,short,fast-paced,fantasy


In [593]:
t['sizes'].value_counts()

medium    287
short      89
long       74
Name: sizes, dtype: int64

In [594]:
t['genre'].value_counts()

fantasy          122
classics         112
sci-fi            77
contemporary      43
horror            27
historical        26
thriller          17
literary          16
romance            8
graphic-novel      1
medium             1
Name: genre, dtype: int64

There is still a value of 'medium' and one of 'graphic-novel', which need to be dealt with, but first I'll drop the unnecessary
columns and combine the other ones with the 'read' dataframe.

In [595]:
t.drop(['c_sizes', 'tone', 'pace'], axis=1, inplace=True)

In [596]:
read.reset_index(drop=True, inplace=True)
t.reset_index(drop=True, inplace=True)
final = pd.concat( [read, t], axis=1) 
final.drop(['Tags'], axis=1, inplace=True)

In [597]:
final

,Title,Authors,Contributors,Read Status,Last Date Read,Moods,Pace,Character- or Plot-Driven?,Strong Character Development?,Loveable Characters?,Diverse Characters?,Flawed Characters?,Star Rating,sizes,genre
0,The Wind-Up Bird Chronicle,Haruki Murakami,Jay Rubin,read,2018-01-01,reflective,slow,Character,No,It's complicated,No,It's complicated,2.50,long,literary
1,The Toll,Neal Shusterman,Missing,read,2019-12-04,adventurous,medium,A mix,Yes,Yes,Yes,Yes,4.00,long,sci-fi
2,Transcendent Kingdom,Yaa Gyasi,Missing,read,2022-04-10,emotional,medium,Character,Yes,Yes,Yes,Yes,4.00,medium,literary
3,Hollow City,Ransom Riggs,Missing,read,2020-03-05,adventurous,medium,A mix,Yes,Yes,Yes,No,3.75,medium,fantasy
4,The Astonishing Color of After,Emily X.R. Pan,Missing,read,2020-01-20,emotional,medium,Character,Yes,Yes,Yes,Yes,4.50,medium,contemporary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,Truly Devious,Maureen Johnson,Missing,read,2020-03-02,mysterious,medium,Plot,Yes,It's complicated,Yes,Yes,3.00,medium,thriller
446,Domniṣoara Christina,Mircea Eliade,Missing,read,2018-01-01,mysterious,medium,Plot,No,It's complicated,No,No,4.00,short,classics
447,Into the Drowning Deep,Mira Grant,Missing,read,2019-01-01,dark,medium,Plot,Yes,Yes,Yes,No,3.75,medium,horror
448,Down Among the Sticks and Bones,Seanan McGuire,Missing,read,2019-01-01,dark,fast,A mix,Yes,Yes,Yes,Yes,5.00,short,fantasy


Rearranging the columns.

In [598]:
final = final[['Title', 'Authors', 'Contributors', 'Read Status', 'Last Date Read', 'Moods', 'Pace', 
               'Character- or Plot-Driven?','Strong Character Development?', 'Loveable Characters?', 'Diverse Characters?',
               'Flawed Characters?', 'sizes', 'genre', 'Star Rating']]
final.loc[final['genre'] == 'medium']

,Title,Authors,Contributors,Read Status,Last Date Read,Moods,Pace,Character- or Plot-Driven?,Strong Character Development?,Loveable Characters?,Diverse Characters?,Flawed Characters?,sizes,genre,Star Rating
96,The Sun and Her Flowers,Rupi Kaur,Missing,read,2019-01-01,emotional,medium,A mix,Yes,Yes,No,Yes,medium,medium,3.0


In [599]:
final.loc[96, 'genre'] = 'contemporary'
final.loc[final['genre'] == 'graphic-novel']

,Title,Authors,Contributors,Read Status,Last Date Read,Moods,Pace,Character- or Plot-Driven?,Strong Character Development?,Loveable Characters?,Diverse Characters?,Flawed Characters?,sizes,genre,Star Rating
272,"The Boy, the Mole, the Fox and the Horse",Charlie Mackesy,Missing,read,2021-01-24,hopeful,fast,Character,It's complicated,Yes,It's complicated,No,short,graphic-novel,4.5


In [600]:
final.loc[272, 'genre'] = 'contemporary'

In [601]:
final.isnull().sum()

Title                            0
Authors                          0
Contributors                     0
Read Status                      0
Last Date Read                   0
Moods                            0
Pace                             0
Character- or Plot-Driven?       0
Strong Character Development?    0
Loveable Characters?             0
Diverse Characters?              0
Flawed Characters?               0
sizes                            0
genre                            0
Star Rating                      0
dtype: int64

In [602]:
final.to_csv('final.csv')